In [946]:
import librosa
import librosa.display
from IPython.display import Audio
import pathlib
from pathlib import Path
import os
import ipywidgets as widgets
from fastai.vision.all import *
import torchvision

In [947]:
path = Path('../')
model_path = path/'models'
model_path.ls()

(#1) [Path('../models/sound_model_V1')]

In [948]:
temp = pathlib.PosixPath
pathlib.PosixPath = pathlib.WindowsPath
learn_inf = load_learner(model_path/'sound_model_V1')
pathlib.PosixPath = temp

In [949]:
btn_upload = widgets.FileUpload(accept=".wav,.mp3")
btn_upload

FileUpload(value={}, accept='.wav,.mp3', description='Upload')

In [950]:
out_pl = widgets.Output()
out_pl.clear_output()
out_pl

Output()

In [951]:
with out_pl: 
    try:
        display(Audio(btn_upload.data[-1]))
    except:
        pass

In [952]:
lbl_pred = widgets.Label()
lbl_pred.value = 'Please select audio'
lbl_pred

Label(value='Please select audio')

In [953]:
btn_run = widgets.Button(description='Classify')
btn_run

Button(description='Classify', style=ButtonStyle())

In [954]:
def log_mel_spec_tfm(audio_upload, dst_path = path/'../data/imgs/uploaded'):
    data, sample_rate = librosa.load(io.BytesIO(audio_upload.data[-1]))
    
    n_fft = 1024
    hop_length = 512
    n_mels = 80
    fmin = 20
    fmax = sample_rate / 2 
    
    mel_spec_power = librosa.feature.melspectrogram(data, sr=sample_rate, n_fft=n_fft, 
                                                    hop_length=hop_length, 
                                                    n_mels=n_mels, power=2.0, 
                                                    fmin=fmin, fmax=fmax)
    
    #mel_spec_power = librosa.feature.melspectrogram(x, sr=sample_rate)
    
    mel_spec_db = librosa.power_to_db(mel_spec_power, ref=np.max)
    
    
    dst_path.mkdir(exist_ok=True)
    fname = list(audio_upload.value)[0]
    
    plt.imsave(dst_path / (fname[:-4] + '.png'), mel_spec_db)
    
    return dst_path / (fname[:-4] + '.png')

In [955]:
def on_click_classify(change):
    out_pl.clear_output()
    with out_pl: display(Audio(btn_upload.data[-1]))
    dst_path = path/'../data/imgs/uploaded'
    audio_img = log_mel_spec_tfm(btn_upload, dst_path)
    pred,pred_idx,probs = learn_inf.predict(audio_img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
    
btn_run.on_click(on_click_classify)

In [956]:
widgets.VBox([widgets.Label('Select your audio'),
      btn_upload, btn_run, out_pl, lbl_pred])